In [1]:
%matplotlib inline

In [2]:
import numpy as np
import tensorflow as tf
from jokerflow import kepler

/Users/dforeman/anaconda/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [17]:
N = 10000000
M = np.random.uniform(0, 1000, N)
e = np.random.uniform(0, 1, N)

In [18]:
M_tensor = tf.convert_to_tensor(M)
e_tensor = tf.convert_to_tensor(e)
E_tensor = kepler(M_tensor, e_tensor)

In [19]:
with tf.Session() as sess:
    E = sess.run(E_tensor)

In [20]:
E.shape

(10000000,)